In [1]:
import os
import math
import imageio
import tensorflow as tf
import random
import matplotlib.pyplot as plt
import numpy as np
import matplotlib as mp
try:
    from tqdm import tqdm
except ImportError:
    def tqdm(x): return x
    print("Install tqdm for cool progress bars")

cwd = os.getcwd()

ModuleNotFoundError: No module named 'imageio'

In [ ]:
# setup

def weight_variable(shape):
    '''
    Initialize weights
    :param shape: shape of weights, e.g. [w, h ,Cin, Cout] where
    w: width of the filters
    h: height of the filters
    Cin: the number of the channels of the filters
    Cout: the number of filters
    :return: a tensor variable for weights with initial values
    '''

    # IMPLEMENT YOUR WEIGHT_VARIABLE HERE
    W = tf.Variable(tf.truncated_normal(shape, stddev=0.1))
    return W

def bias_variable(shape):
    '''
    Initialize biases
    :param shape: shape of biases, e.g. [Cout] where
    Cout: the number of filters
    :return: a tensor variable for biases with initial values
    '''

    # IMPLEMENT YOUR BIAS_VARIABLE HERE
    b = tf.Variable(tf.constant(0.0, shape=shape))
    return b

def conv2d(x, W):
    '''
    Perform 2-D convolution
    :param x: input tensor of size [N, W, H, Cin] where
    N: the number of images
    W: width of images
    H: height of images
    Cin: the number of channels of images
    :param W: weight tensor [w, h, Cin, Cout]
    w: width of the filters
    h: height of the filters
    Cin: the number of the channels of the filters = the number of channels of images
    Cout: the number of filters
    :return: a tensor of features extracted by the filters, a.k.a. the results after convolution
    '''

    # IMPLEMENT YOUR CONV2D HERE
    h_conv = tf.nn.conv2d(x, W, strides[1, 1, 1, 1], padding='SAME')
    return h_conv

def max_pool_2x2(x):
    '''
    Perform non-overlapping 2-D maxpooling on 2x2 regions in the input data
    :param x: input data
    :return: the results of maxpooling (max-marginalized + downsampling)
    '''

    # IMPLEMENT YOUR MAX_POOL_2X2 HERE
    h_max = tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    return h_max

In [ ]:
# helpers
def var_log(name, vector):
    with tf.name_scope(name + "_SUMMARY"):
        mean, var = tf.nn.moments(vector, axes=list(range(tf.rank(vector).eval())))
        tf.summary.scalar("mean", mean)
        tf.summary.scalar("variance", var)
        tf.summary.scalar("std", tf.sqrt(var))
        tf.summary.scalar("max", tf.reduce_max(vector))
        tf.summary.scalar("min", tf.reduce_min(vector))
        tf.summary.histogram('Hist', vector)

In [2]:
ntrain = 1000 # per class
ntest = 100 # per class
nclass = 10 # number of classes
imsize = 28
nchannels = 1
batchsize = 128
epochs = 50

Train = np.zeros((ntrain*nclass,imsize,imsize,nchannels))
Test = np.zeros((ntest*nclass,imsize,imsize,nchannels))
LTrain = np.zeros((ntrain*nclass,nclass))
LTest = np.zeros((ntest*nclass,nclass))

itrain = -1
itest = -1
for iclass in range(0, nclass):
    for isample in range(0, ntrain):
        path = '~/CIFAR10/Train/%d/Image%05d.png' % (iclass,isample)
        im = img.imread(path) # 28 by 28
        im = im.astype(float)/255
        itrain += 1
        Train[itrain,:,:,0] = im
        LTrain[itrain,iclass] = 1 # 1-hot label
    for isample in range(0, ntest):
        path = '~/CIFAR10/Test/%d/Image%05d.png' % (iclass,isample)
        im = img.imread(path) # 28 by 28
        im = im.astype(float)/255
        itest += 1
        Test[itest,:,:,0] = im
        LTest[itest,iclass] = 1 # 1-hot label

sess = tf.InteractiveSession()

tf_data = tf.placeholder(tf.float32, shape=[None, imsize, imsize, nchannels])#tf variable for the data, remember shape is [None, width, height, numberOfChannels]
tf_labels = tf.placeholder(tf.float32, shape=[None, nclass])#tf variable for labels

NameError: name 'np' is not defined

In [ ]:
# --------------------------------------------------
# model
#create your model
x = tf_data
W1 = weight_variable([5, 5, 1, 32])
b1 = bias_variable([32])
h1 = tf.nn.relu(conv2d(x, W1) + b1) #Running it with ReLu activation
h_pool1 = max_pool_2x2(h1)

var_log("W1", W1)
var_log("b1", b1)
var_log("h1", h1)
var_log("h_pool1", h_pool1)

h_pool1_dropoff = tf.nn.dropout(h_pool1, tf.placeholder(tf.float32))

W2 = weight_variable([5, 5, 64, 128])
b2 = bias_variable([128])
h2 = tf.nn.relu(conv2d(x, W2) + b2) #Running it with ReLu activation
h_pool2 = max_pool_2x2(h2)

var_log("W2", W2)
var_log("b2", b2)
var_log("h2", h2)
var_log("h_pool2", h_pool2)

h_pool2_dropoff = tf.nn.dropout(h_pool2, tf.placeholder(tf.float32))

conv_out = h_pool2_dropoff

out_shape = conv_out.get_shape().as_list()
flat_shape = out_shape[1] * out_shape[2] * out_shape[3]
flat_conv = tf.reshape(conv_out, [-1, flat_shape])

prob = tf.placeholder(tf.float32)
flat_conv_dropout = tf.nn.dropout(flat_conv, prob)

# final layer
W_full2 = weight_variable([flat_shape, 10])
b_full2 = bias_variable([10])
y_conv = tf.matmul(flat_conv_dropout, W_full2) + b_full2

var_log("W_full2", W_full2)
var_log("b_full2", b_full2)
var_log("y_conv", y_conv)

In [ ]:
# --------------------------------------------------
# loss
# set up the loss, optimization, evaluation, and accuracy
# setup training
# --------------------------------------------------
cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_labels, logits=y_conv))
opt = tf.train.AdamOptimizer(1e-3) # MODIFIED this learning rate as a variable for accuracy.
optimizer = opt.minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(tf_labels, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

tf.summary.scalar("Loss", cross_entropy)
tf.summary.scalar("Accuracy", accuracy)